In [1]:
import numpy as np

In [139]:
grid = np.array([[0,0,0,0,0,0,0,0,0],
                    [0,1,1,1,1,2,0,0,0],
                    [0,2,2,2,0,2,2,0,0],
                    [0,0,0,0,0,0,0,2,0],
                    [0,0,2,2,2,2,0,2,0],
                    [0,0,0,0,0,0,0,2,0],
                    [0,2,2,2,0,2,2,0,0],
                    [0,0,0,0,0,0,0,0,3],
                    [0,1,1,1,2,1,1,0,2]]) 
rewards = {0:-1, 1:5, 2:-20, 3:100}
actions = np.array([[1, 1/8],[2, 1/8], [3, 5/8],[4, 1/8]]) # up, right, down, left
gama = 0.9
threshold = 0.01
n =9
states = n*n
V = np.zeros(states)
Vnew = np.zeros(states)
iterate=True

def get_next_state(row,col,a):
    # going up
    if a == 1:
        # if state is in row 0 it stays in same row
        if row == 0:
            r = row
            c = col
            reward = -10
        else:
            # if next state is not reachable
            reward = rewards[grid[row-1,col]]
            if grid[row-1,col] == 2:
                r = row
                c = col
            # if next state is reachable and no obstacle
            else:
                r = row-1
                c = col
    # go right
    elif a == 2:
        # if state is in last column it stays in same column
        if col == 8:
            r = row
            c = col
            reward = -10
        else:
            # if next state is not reachable
            reward = rewards[grid[row,col+1]]
            if grid[row,col+1] == 2:
                r = row
                c = col
            # if next state is reachable and no obstacle
            else:
                r = row
                c = col+1
    # go down
    elif a == 3:
        # if state is in row 8 it stays in same row
        if row == 8:
            r = row
            c = col
            reward = -10
        else:
            # if next state is not reachable
            reward = rewards[grid[row+1,col]]
            if grid[row+1,col] == 2:
                r = row
                c = col
                reward = rewards[grid[r+1,c]]
            # if next state is reachable and no obstacle
            else:
                r = row+1
                c = col
                reward = rewards[grid[r,c]]
    # go left
    elif a == 4:
        # if state is in first column it stays in same column
        if col == 0:
            r = row
            c = col
            reward = -10
        else:
            # if next state is not reachable
            reward = rewards[grid[row,col-1]]
            if grid[row,col-1] == 2:
                r = row
                c = col
            # if next state is reachable and no obstacle
            else:
                r = row
                c = col-1
    return r,c,reward

def get_value(row,col,V):
    index = row*9 + col
    return V[index]


# for count in range(0,10):
count = 0
iterate = True
while iterate:
    print(count)
    delta = 0
    for i in range(states):
#         print(i)
        row = i % n
        col = int(i / n)
        s = grid[row,col]
        if s != 2 and s != 3:
            for j in range(1,4): # for each action
                r,c,reward = get_next_state(row,col,j) # get next state and reward
                next_v = get_value(r,c,V) # get value from nest state
                prob = actions[j-1,1] # probability of action
                Vnew[i]  += prob*(reward + gama*next_v)
#             print('vnew ' + str(Vnew[i]) + ' vold ' + str(V[i]) )
            delta = max(delta, np.abs(V[i] - Vnew[i]))
#             print('delta = ' + str(delta) + ' v = ' + str(np.abs(V[i] - Vnew[i])))
    V[:]  = Vnew[:]
    count += 1
    if delta < threshold:
        break
    if count > 1:
        break

V.shape = (n,n)

0
1


In [142]:

print(count)
print(delta)
print(V.T)

2
54.7546875
[[  -3.2546875   -3.6296875    3.0359375    1.475        1.475      -28.70625
    -4.4640625   -4.4640625  -10.778125 ]
 [  -0.840625   -30.7640625  -24.73125    -26.2921875   -6.9640625    0.
   -25.809375    -2.2140625   -8.528125 ]
 [  -5.121875     0.           0.           0.          -5.365625     0.
     0.         -25.809375   -10.2859375]
 [  -1.91875     -9.678125   -31.9515625  -40.459375   -27.2015625
   -40.459375   -13.078125     0.          -5.6453125]
 [  -9.7234375   -8.215625     0.           0.           0.           0.
    -6.865625     0.          -8.2328125]
 [  -2.678125   -25.865625   -31.9515625  -40.459375   -10.0296875
   -40.459375   -30.615625     0.          -8.303125 ]
 [  -9.4109375    0.           0.           0.          -6.865625     0.
     0.          -0.565625   115.8796875]
 [  -3.83125     -1.6578125   -1.559375    -0.125      -25.865625
    -0.490625    36.3390625   22.5578125    0.       ]
 [ -13.7078125  -12.9484375  -14.284375   